In [27]:
from cv2 import mean
import numpy as np
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import argparse
import os
import torch
from datetime import datetime
import math
from model.LSnet import * 
import neptune.new as neptune
from sklearn.metrics import mean_squared_error , mean_absolute_percentage_error, mean_absolute_error

## Write testset to .txt

* FORMAT  
육안검사, 코로나검사, 적외선검사, PD검사, 문진점수, 기준수명, 경과수명, HI, 최종수명 

    ex) 100,100,75,100,30,30,20,85.19999999999999,8.52

Or use utils/gendataset.py to generate test dataset.

In [77]:
os.makedirs("Dataset/", exist_ok= True)
test_list = np.array([75,50,74,0,10,30,27,38.008,1])
f = open("Dataset/inference.txt",'w')
# f.write(test_list)
f.write(",".join(str(j) for j in test_list)+"\n")
f.close()

In [85]:
class LSDataset_val(Dataset):
    def __init__(self):
        txt ='Dataset/inference.txt'
        self.f_ = np.loadtxt(txt, 'str', delimiter=',', skiprows=0)
    
    def __len__(self):
        return self.f_.shape[0]
    
    def __getitem__(self, idx):
        raw = self.f_
        data = torch.Tensor(raw[:-2].astype("float"))
        hi = torch.Tensor([raw[-2].astype("float")])
        flsp=torch.Tensor([raw[-1].astype("float")])
        return data, hi, flsp 

In [86]:
test_dataset= LSDataset_val()
test_loader = DataLoader(dataset= test_dataset, batch_size = 1, shuffle = False, num_workers=0)
print(test_dataset[0])

(tensor([100., 100.,  75., 100.,  30.,  30.,  20.]), tensor([85.2000]), tensor([8.5200]))


## Model Config

In [87]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
batch_size =1
test_dataset= LSDataset_val()
test_loader = DataLoader(dataset= test_dataset, batch_size = 1, shuffle = False, num_workers=0)
model = Build14(in_ch=7, out_ch=1)
model.load_state_dict(torch.load('/mnt/e/Workspace/jm/Projects/baby_eval/output/SL1_SGD_300_0010414.pth')) # model path
model.to(device)
model.eval()
criterion = [torch.nn.SmoothL1Loss(), torch.nn.SmoothL1Loss()]

In [88]:
cnt=0
for epoch in range(1):
    iter = math.ceil(len(test_dataset)/batch_size)
    loss_val, mae_hi, mape_hi, mae_fnls, mape_fnls =0,0,0,0,0
    cnt+=1
    with torch.no_grad():
        for data, hi_gt, fnls_gt in test_loader:
            x, y1_val, y2_val = data.to(device), hi_gt.to(device), fnls_gt.to(device)
            y1_pred_val, y2_pred_val = model(x)                
            val_loss_hi = criterion[0](y1_pred_val, y1_val)
            val_loss_fnls = criterion[1](y2_pred_val, y2_val)
            val_loss =   val_loss_hi +  val_loss_fnls

            hi_pred = y1_pred_val.squeeze().cpu().numpy()
            fnls_pred = y2_pred_val.squeeze().cpu().numpy()
            hi_gt = y1_val.squeeze().cpu().numpy() 
            fnls_gt = y2_val.squeeze().cpu().numpy() 

            mae_hi += np.abs(hi_gt- hi_pred)
            mape_hi += 100* np.abs(hi_gt-hi_pred)/hi_gt
            mae_fnls +=  np.abs(fnls_gt- fnls_pred)
            mape_fnls += 100* np.abs(fnls_gt-fnls_gt)/fnls_gt
print("HI pred: {},  HI GT: {}\nLS pred: {}, LS GT: {}".format(hi_pred, hi_gt, fnls_pred, fnls_gt))
print("mae : {}, {}".format(mae_hi, mae_fnls))
print("mape : {}, {}".format(mape_hi, mape_fnls))

# mae = mean_squared_error(hi_gt, hi_pred)

HI pred: 85.22245788574219,  HI GT: 85.19999694824219
LS pred: 8.796286582946777, LS GT: 8.520000457763672
mae : 0.2021484375, 2.486575126647949
mape : 0.23726343279425513, 0.0
